In [11]:
%pip install --upgrade Pillow

/Users/dhruvanand/miniforge3/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [13]:
import requests
from transformers import ResNetModel, ConvNextImageProcessor

import torch
from PIL import Image

model = ResNetModel.from_pretrained("microsoft/resnet-50")
# embed https://upload.wikimedia.org/wikipedia/commons/8/8b/Scolopendra_gigantea.jpg
image_url = "/Users/dhruvanand/Code/vector-io/Scolopendra_gigantea.jpg"


# Load the image as a torch tensor

image_processor = ConvNextImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")
img = Image.open(image_url)
inputs = image_processor(images=img, return_tensors="pt")
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([1, 2048, 7, 7])


In [14]:
# apply GMP over the outputs.hidden_states[-1] which is of shape (1, 2048, 7, 7)
# to get a single vector of shape (1, 2048)
pooled_output = outputs.last_hidden_state.max(dim=2).values.max(dim=2).values

print(pooled_output.shape)

torch.Size([1, 2048])


In [15]:
pooled_output

tensor([[0.0637, 2.6114, 0.0000,  ..., 0.1222, 1.0969, 0.0000]],
       grad_fn=<MaxBackward0>)

In [3]:
%pip install flax

  Obtaining dependency information for flax from https://files.pythonhosted.org/packages/8d/4a/7e78abc8392ff21b0257deb79e842f80647b63b745447df94893732d60fd/flax-0.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for jax>=0.4.19 from https://files.pythonhosted.org/packages/35/c1/51a51ea646864ef8b4bd68632162ed1b4495c580b9bfa72fe49168221d69/jax-0.4.24-py3-none-any.whl.metadata
  Obtaining dependency information for msgpack from https://files.pythonhosted.org/packages/ad/72/d39ed43bfb2ec6968d768318477adb90c474bdc59b2437170c6697ee4115/msgpack-1.0.7-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for optax from https://files.pythonhosted.org/packages/bb/c6/ca58ae4ed7283c65ab53318096cd53a3ab78462d0488a185eef3cf2f2d04/optax-0.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for orbax-checkpoint from https://files.pythonhosted.org/packages/83/a2/0677f2ee06bdbf7b4e6be4ad931ffe58f2ea82d67bb2a277d9d7b3b1e352/orbax_checkpoint-0.5.3-py3-n

In [1]:
from PIL import Image
from transformers import ConvNextFeatureExtractor, FlaxResNetModel

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image_processor = ConvNextFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = FlaxResNetModel.from_pretrained("microsoft/resnet-50")
inputs = image_processor(images=image, return_tensors="np")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

/Users/dhruvanand/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/dhruvanand/miniforge3/lib/python3.10/site-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(
flax_model.msgpack: 100%|██████████| 102M/102M [00:06<00:00, 15.1MB/s] 


In [2]:
last_hidden_states.shape

(1, 2048, 7, 7)

In [5]:
import torch.nn as nn


gap = nn.AdaptiveAvgPool2d((1, 1))
x_reduced = gap(last_hidden_states)

# Remove extra dimensions to get a (1, 2048) vector
x_reduced = torch.flatten(x_reduced, start_dim=1)

print(x_reduced.shape)

TypeError: 'int' object is not callable